In [25]:
%pip install numpy tensorflow scikit-learn 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()


In [27]:
# 2. Normalizar os dados (0-1)
# Essa etapa é importante para garantir que os valores dos pixels estejam na mesma escala (0 a 1),
# o que ajuda a melhorar a performance e a estabilidade do treinamento do modelo.
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# 3. Achatar imagens para usar em MLP (28x28 → 784)
# Isso é necessário porque redes MLP (Perceptron Multicamadas) esperam vetores unidimensionais como entrada,
# então precisamos transformar as imagens 2D (28x28) em vetores 1D (784 elementos).
x_train = x_train.reshape(-1, 28 * 28)
x_test = x_test.reshape(-1, 28 * 28)

# 4. Embaralhar os dados
# Embaralhar os dados é importante para garantir que a ordem dos exemplos de treinamento
# não introduza viés durante o treinamento do modelo.
x_train, y_train = shuffle(x_train, y_train, random_state=42)

In [28]:
# 5. Dividir em 5 clientes
num_clients = 5
client_data = {}
samples_per_client = x_train.shape[0] // num_clients

for i in range(num_clients):
    start = i * samples_per_client
    end = (i + 1) * samples_per_client
    client_data[f'cliente_{i+1}'] = {
        'x': x_train[start:end],
        'y': y_train[start:end]
    }

# 6. Verificação rápida
for client, data in client_data.items():
    print(f"{client}: {data['x'].shape[0]} amostras")

cliente_1: 12000 amostras
cliente_2: 12000 amostras
cliente_3: 12000 amostras
cliente_4: 12000 amostras
cliente_5: 12000 amostras


In [29]:
# 7. Treinamento do modelo
# Aqui, criamos um modelo simples de rede neural com uma camada oculta e uma camada de saída.
# O modelo é compilado com o otimizador Adam e a função de perda de entropia cruzada esparsa.
def criar_modelo():
    modelo = Sequential([
        Dense(128, activation='relu', input_shape=(784,)),
        Dense(10, activation='softmax')
    ])
    modelo.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])
    return modelo


In [30]:
# 8. Treinamento local em cada cliente
# Cada cliente treina seu modelo localmente com seus próprios dados
def treinar_localmente(modelo, x_dados, y_dados, epochs=5, batch_size=32):
    modelo.fit(x_dados, y_dados, epochs=epochs, batch_size=batch_size, verbose=0)
    return modelo.get_weights()


In [31]:
# 9. Agregação dos pesos
# A função `fed_avg` calcula a média dos pesos dos modelos treinados localmente pelos clientes.
def fed_avg(pesos_clientes):
    # pesos_clientes: lista de listas de arrays (pesos de cada cliente)
    num_clientes = len(pesos_clientes)
    media = []
    
    for camadas in zip(*pesos_clientes):
        camada_media = np.mean(np.array(camadas), axis=0)
        media.append(camada_media)
    
    return media

In [32]:
# Parâmetros
rodadas = 5
epochs_local = 1
modelo_global = criar_modelo()

for r in range(rodadas):
    print(f"\n📡 Rodada Federada {r+1}")

    pesos_clientes = []

    # Etapa 1: cada cliente treina localmente
    # Cada cliente treina seu modelo localmente com seus próprios dados
    # e retorna os pesos treinados para o servidor.
    # O servidor então agrega esses pesos para atualizar o modelo global.
    for nome_cliente, dados in client_data.items():
        modelo_local = criar_modelo()
        modelo_local.set_weights(modelo_global.get_weights())
        
        pesos_treinados = treinar_localmente(
            modelo_local,
            dados['x'],
            dados['y'],
            epochs=epochs_local
        )
        pesos_clientes.append(pesos_treinados)
        print(f"{nome_cliente} treinado.")

    # Etapa 2: servidor faz agregação FedAvg
    # O servidor agrega os pesos dos modelos locais para criar um modelo global.
    # Isso é feito calculando a média dos pesos de cada camada dos modelos locais.
    # A média é calculada para cada camada, resultando em um novo conjunto de pesos para o modelo global.
    # O modelo global é então atualizado com esses pesos agregados.
    pesos_agrupados = fed_avg(pesos_clientes)
    modelo_global.set_weights(pesos_agrupados)
    print("🧠 Modelo global atualizado.")



📡 Rodada Federada 1


c:\Users\beatr\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


cliente_1 treinado.
cliente_2 treinado.
cliente_3 treinado.
cliente_4 treinado.
cliente_5 treinado.
🧠 Modelo global atualizado.

📡 Rodada Federada 2
cliente_1 treinado.
cliente_2 treinado.
cliente_3 treinado.
cliente_4 treinado.
cliente_5 treinado.
🧠 Modelo global atualizado.

📡 Rodada Federada 3
cliente_1 treinado.
cliente_2 treinado.
cliente_3 treinado.
cliente_4 treinado.
cliente_5 treinado.
🧠 Modelo global atualizado.

📡 Rodada Federada 4
cliente_1 treinado.
cliente_2 treinado.
cliente_3 treinado.
cliente_4 treinado.
cliente_5 treinado.
🧠 Modelo global atualizado.

📡 Rodada Federada 5
cliente_1 treinado.
cliente_2 treinado.
cliente_3 treinado.
cliente_4 treinado.
cliente_5 treinado.
🧠 Modelo global atualizado.


In [33]:
# Avaliar no conjunto de teste
loss_fed, acc_fed = modelo_global.evaluate(x_test, y_test, verbose=0)
print(f"Acurácia: {acc_fed*100:.2f}%")
print(f"Perda (Loss): {loss_fed:.4f}")



Acurácia: 85.90%
Perda (Loss): 0.3962


In [34]:
# Criar novo modelo centralizado
modelo_centralizado = criar_modelo()

# Treinar com todos os dados de treino
modelo_centralizado.fit(x_train, y_train, epochs=5, batch_size=32, verbose=1)

# Avaliar no conjunto de teste
loss_central, acc_central = modelo_centralizado.evaluate(x_test, y_test, verbose=0)
print(f"\n🏁 Acurácia do modelo centralizado: {acc_central * 100:.2f}%")
print(f"Perda (Loss): {loss_central:.4f}")


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7757 - loss: 0.6401
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8617 - loss: 0.3809
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8757 - loss: 0.3466
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8852 - loss: 0.3121
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8946 - loss: 0.2916

🏁 Acurácia do modelo centralizado: 86.99%
Perda (Loss): 0.3709
